In [1]:
##ここだけ設定する---------------------------------------------------------------------
##フレーズを先に指定する
phrase = 'そんしつにめげないこうみん' #プロンプト文 f
# phrase = 'おおどおりにめんしたまますいみん' #プロンプト文 b
# phrase = 'のびるさんすいほーすはそんあいつぐせつびふび' #プロンプト文 a
##-----------------------------------------------------------------------------------

In [2]:
import os, glob
import librosa
# from librosa import core
import soundfile as sf
import pandas as pd
import subprocess

import tkinter as tk
from tkinter import filedialog

#実行結果の「.lab」ファイルと元の音源(44.1kHz)からPNデータを導出し、DB（excel）に記録する。
import shutil
import my_func3 #自作のPND関数を呼び出す


#フォルダ名を取得
root = tk.Tk()# ルートウィンドウ作成
root.withdraw()# ルートウィンドウの非表示
idir =  './' #カレントディレクトリ
dir_path = tk.filedialog.askdirectory(initialdir = idir)
# dir_path = '../../音源/f/prompt_f_sonsitsu/'
dir_list = os.listdir(dir_path)
print(dir_list)



c:\Users\kktak\anaconda3\envs\myenv\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


['kitano']


In [3]:

#PNデータ格納用
pho_PNene = pd.DataFrame()


#各フォルダから音声データ（.wav）のpathを入手
for dir_name in dir_list:
    # 全てのwavファイルのパスをpathlistに格納 / **の部分があらゆる中間ディレクトリに対応
    pathlist = glob.glob(dir_path+'/'+ dir_name + '/**/*.wav', recursive=True)
    
    #各wav音声ファイルに対応するテキストファイルを作成する / wavファイルをJulius用にダウンサンプリング（44.1kHz　to 16kHz）する
    for path_name in pathlist:
        print(path_name)
        file_name = os.path.splitext(os.path.basename(path_name))[0]
        print(file_name)

        #テキストファイルをまとめて入れるためのフォルダを作成する
        new_txt_path = 'wav/{}.txt'.format(file_name)
        f = open(new_txt_path, 'w', encoding = 'utf8')
        f.write('{}'.format(phrase)) #日本語フレーズをテキストに記入
        f.close()

        # サンプリングレート44.1kHzのwavを16kHzに変換する
        y, sr = librosa.core.load(path_name, sr=16000, mono=False) # 16kHz、モノラル(1ch)ではなくステレオ（2ch）で読み込み
        new_wav_path = 'wav/{}.wav'.format(file_name)
        y = y[0] #こっちがclean音声なので、segmentationにわたす
        sf.write(new_wav_path, y, sr, subtype="PCM_16") #16bitで書き込み
        
    #Julius-segmentationを実行
    subprocess.run(['perl','segment_julius.pl'], stdout = subprocess.PIPE, stderr = subprocess.PIPE,shell = True)


    # 全てのwavファイルのパスをpathlistに格納 / **の部分があらゆる中間ディレクトリに対応
    pathlist = glob.glob(dir_path+'/' + dir_name + '/**/*.wav', recursive=True)
    
    for path_name in pathlist:
        path_name = path_name.replace('\\','/')
        print(f"deir_name = {dir_name}")
        print(f"deir_name = {path_name}")
        result = my_func3.pnd(dir_name,path_name)
        pho_PNene = pd.concat([pho_PNene,result], axis=0)

    #処理中に作成したwavファイルを全て削除する →　新しいwavフォルダの作成
    shutil.rmtree('wav')
    os.mkdir('wav')


#前回のoutputフォルダをクリア → 新しいフォルダの作成
shutil.rmtree('output') 
os.mkdir('output') 
pho_PNene.to_excel('output/output.xlsx',index=False, header=pho_PNene.columns)



C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20211029\201110.wav
201110
C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20211029\201111.wav
201111
C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20220617\220617_001-01.wav
220617_001-01
C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20220617\220617_001-02.wav
220617_001-02
C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20220617\220617_001-03.wav
220617_001-03
C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20220617\220617_001-04.wav
220617_001-04
C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20220617\220617_001-05.wav
220617_001-05
C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20220617\220617_001-06.wav
220617_001-06
C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20220617\220617_001-07.wav
220617_001-07
C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20220617\220617_001-08.wav
220617_001-08
C:/Users/kktak/OneDrive/Desktop/pronpt_f_others/kitano\20220617\2206

c:\Users\kktak\anaconda3\envs\myenv\lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] 指定されたファイルが見つかりません。

In [ ]:
#wavフォルダのリセット用
import shutil

shutil.rmtree('wav')
os.mkdir('wav')